In [1]:
# @Author: xie
# @Date:   2025-06-20
# @Email:  xie@umd.edu
# @Last modified by:   xie
# @Last modified time: 2025-06-20
# @License: MIT License

'''
GeoRF_main.py

This file shows an example on how to define, train and evaluate a GeoRF model.
The general usage is similar to standard machine learning models, with fit(), predict() and evaluate() functions.
The main difference is that GeoRF is a spatially-explicit model, which means location info is needed as part of inputs.

Usage:
		There are 4 key steps:
			(1) Data loading: Prepare standard X, y, as well as locations X_loc
			(2) Define groups using GroupGenerator() or by customizing it for your own data: This is needed in GeoRF, which defines the minimum spatial unit for space partitioning.
					For example, a grid can be used to split the study area into groups, where all data points in each grid cell form a group.
					The data may contain thousands of groups, which may be partitioned by GeoRF into 10-20 partitions (one RF per partition). This is just to give a sense of the quantities.
					In testing, the grouping is used to assign test data points to local models (one local RF model is learned for each partition).
			(3) Training: georf.fit()
			(4) Prediction and evaluation: georf.predict() and georf.evaluate()

		For visualizations, code for grid-based grouping is provided. For other group definitions, please write customized visualization functions.
'''

import numpy as np
from scipy import stats
import pandas as pd
import os
import argparse
import sys
#GeoRF
#Can be customized with the template
from GeoRF import GeoRF
from customize import *
from data import load_demo_data#load_data_us_cdl
from helper import get_spatial_range
from initialization import train_test_split_all
#All global parameters
from config import *
from sklearn.impute import SimpleImputer
import polars as pl
# import machine learning metrics, precision, recall and f1_score
from sklearn.metrics import precision_score, recall_score, f1_score

data = pl.read_csv(r"C:\Users\swl00\IFPRI Dropbox\Weilun Shi\Google fund\Analysis\1.Source Data\FEWSNET_IPC_train_lag_forecast_v06252025.csv")
# drop ISO3, fews_ipc_adjusted, fews_proj_med_adjusted, fews_ipc,fews_proj_near, fews_proj_near_ha, fews_proj_med, fews_proj_med_ha, ADMIN0, ADMIN1,ADMIN2,ADMIN3,ISO



In [2]:
# customize function
def comp_impute(X, strategy = "max_plus", multiplier = 100.0):
    # loop over X columns, replace inf with missing
    for _ in range(X.shape[1]):
        print(f"Checking column {_} for inf values")
        # Try to convert column to float, ignore errors (non-convertible values remain unchanged)
        col_data = X[:, _]
        try:
            col_data_float = col_data.astype(float)
            if np.isinf(col_data_float).any():
                print(f"Column {_} has inf values, replacing with NaN")
                col_data_float[np.isinf(col_data_float)] = np.nan
                X[:, _] = col_data_float
        except Exception as e:
            print(f"Column {_} could not be converted to float: {e}")
            # Skip columns that cannot be converted
            continue
    X_imputed, imputer = impute_missing_values(X, strategy=strategy, multiplier=multiplier, verbose=True)
    X = X_imputed
    return X

In [ ]:
# determine xmin,xmax,ymin and ymax first before runing model

In [3]:
# Data Preprocessing
cols_to_drop = [
        "ISO3", "fews_ipc_adjusted", "fews_proj_med_adjusted", "fews_ipc",
        "fews_proj_near", "fews_proj_near_ha", "fews_proj_med",
        "fews_proj_med_ha", "ADMIN0", "ADMIN1", "ADMIN2", "ADMIN3"
    ]
data = data.drop(cols_to_drop)
less_var = True
if less_var:
    var_extra = ['Evap_tavg_mean',
    'Qsb_tavg_mean',
    'RadT_tavg_mean',
    'SnowCover_inst_mean',
    'SnowDepth_inst_mean',
    'Snowf_tavg_mean',
    'SoilMoi00_10cm_tavg_mean',
    'SoilMoi10_40cm_tavg_mean',
    'SoilMoi100_200cm_tavg_mean',
    'SoilMoi40_100cm_tavg_mean',
    'LWdown_f_tavg_mean',
    'SoilTemp00_10cm_tavg_mean',
    'SoilTemp10_40cm_tavg_mean',
    'SoilTemp100_200cm_tavg_mean',
    'SoilTemp40_100cm_tavg_mean',
    'SWdown_f_tavg_mean',
    'SWE_inst_mean',
    'Swnet_tavg_mean',
    'Wind_f_tavg_mean',
    'Lwnet_tavg_mean',
    'Psurf_f_tavg_mean',
    'Qair_f_tavg_mean',
    'Qg_tavg_mean',
    'Qh_tavg_mean',
    'Qle_tavg_mean',
    'Qs_tavg_mean']
    # drop any columns start with var_extra
    data = data.drop([col for col in data.columns if col.startswith(tuple(var_extra))])
    


# keep only fews_ipc_crisis is not null
data = data.filter(pl.col("fews_ipc_crisis").is_not_null())

# encode ISO
data = data.with_columns([
    pl.col("ISO").cast(pl.Categorical).to_physical().alias("ISO_encoded")
])

# drop unit name, ISO
data = data.drop(["unit_name", "ISO"])
# for AEZ_* replace "False" with 0 and "True" with 1
for col in data.columns:
    if col.startswith("AEZ_"):
        data = data.with_columns(
            pl.when(pl.col(col) == "True")
            .then(1)
            .when(pl.col(col) == "False")
            .then(0)
            .otherwise(pl.col(col))
            .alias(col)
        )


df = data.to_pandas()
df['date'] = pd.to_datetime(df['date'])
df['years'] = df['date'].dt.year
df['fews_ipc_crisis_lag_1'] = df.groupby('FEWSNET_admin_code')['fews_ipc_crisis'].shift(1)
df['fews_ipc_crisis_lag_2']= df.groupby('FEWSNET_admin_code')['fews_ipc_crisis'].shift(2)
df['fews_ipc_crisis_lag_3'] = df.groupby('FEWSNET_admin_code')['fews_ipc_crisis'].shift(3)
# generate dummys for year month
for year in df['years'].unique():
    df[f'year_{year}'] = (df['years'] == year).astype(int)
for month in df['date'].dt.month.unique():
    df[f'month_{month}'] = (df['date'].dt.month == month).astype(int)




X_loc = df[['lat', 'lon']].values



aez_columns = [col for col in df.columns if col.startswith('AEZ_')]
df['AEZ_group'] = df.groupby(aez_columns).ngroup()
df['AEZ_country_group'] = df.groupby(['AEZ_group', 'ISO_encoded']).ngroup()

In [4]:

#prepare groups (minimum spatial units for space-partitioning and location groupings, e.g., using a grid)
assignment = 'all_kmeans' # if polygons is True, use FEWSNET_admin_code as groups, otherwise use grid-based grouping
#extract the values of FEWSNET_admin_code and assign to X_group
if assignment=='polygons':
    X_group = df['FEWSNET_admin_code'].values
    X_group = X_group.astype(int)
elif assignment=='grid':
    #prepare groups (minimum spatial units for space-partitioning and location groupings, e.g., using a grid)
    xmin, xmax, ymin, ymax = get_spatial_range(X_loc)
    STEP_SIZE = 0.1
    group_gen = GroupGenerator(xmin, xmax, ymin, ymax, STEP_SIZE)
    X_group = group_gen.get_groups(X_loc)
elif assignment=='country':
    X_group = df['ISO_encoded'].values
    X_group = X_group.astype(int)
elif assignment=='AEZ':
    X_group = df['AEZ_group'].values
    X_group = X_group.astype(int)
elif assignment=='country_AEZ':
    X_group = df['AEZ_country_group'].values
    X_group = X_group.astype(int)
elif assignment=='geokmeans':
    cluster_id, cluster_info, admin_to_group_map = create_kmeans_groupgenerator_from_admin_codes(df, n_clusters=100, random_state=42, features_for_clustering=['lat', 'lon'])
    X_group=cluster_id
    X_group = X_group.astype(int)
    # from admin_to_group_map, create a correspondence table, its index is FEWSNET_admin_code, and column is cluster_id
    correspondence_table = pd.DataFrame(list(admin_to_group_map.items()), columns=['FEWSNET_admin_code', 'cluster_id'])
    correspondence_table.to_csv('correspondence_table_geokmeans.csv', index=False)
elif assignment=='all_kmeans':
    X_columns = df.drop(columns=['FEWSNET_admin_code', 'lat', 'lon', 'fews_ipc_crisis']).columns.tolist()
    cluster_id, cluster_info, admin_to_group_map = create_kmeans_groupgenerator_from_admin_codes(df, n_clusters=100, random_state=42, features_for_clustering=X_columns)
    X_group = cluster_id
    X_group = X_group.astype(int)
    correspondence_table = pd.DataFrame(list(admin_to_group_map.items()), columns=['FEWSNET_admin_code', 'cluster_id'])
    correspondence_table.to_csv('correspondence_table_allkmeans.csv', index=False)
else:
    raise ValueError(f"Unknown assignment method: {assignment}")
y = df['fews_ipc_crisis'].values


# save correspondence table
correspondence_table = df[['FEWSNET_admin_code', 'AEZ_group', 'AEZ_country_group', 'ISO_encoded']].drop_duplicates()
correspondence_table.to_csv('correspondence_table.csv', index=False)
#X impute with column mean




# sort by FEWSNET_admin_code and date
df = df.sort_values(by=['FEWSNET_admin_code', 'date'])
# for each FEWSNET_admin_code, created lagged fews_ipc_crisis for 1,2 and 3 terms
for lag in range(1, 4):
    df[f'fews_ipc_crisis_lag_{lag}'] = df.groupby('FEWSNET_admin_code')['fews_ipc_crisis'].shift(lag)
# drop date
df = df.drop(columns=['date'])
df=df.drop(columns=['fews_ipc_crisis','AEZ_group','ISO_encoded','AEZ_country_group'])
time_variants =  ['event_count_battles', 'event_count_explosions', 'event_count_violence', 'sum_fatalities_battles', 'sum_fatalities_explosions', 'sum_fatalities_violence', 'event_count_battles_w5', 'event_count_explosions_w5', 'event_count_violence_w5', 'sum_fatalities_battles_w5', 'sum_fatalities_explosions_w5', 'sum_fatalities_violence_w5', 'event_count_battles_w10', 'event_count_explosions_w10', 'event_count_violence_w10', 'sum_fatalities_battles_w10', 'sum_fatalities_explosions_w10', 'sum_fatalities_violence_w10', 'nightlight', 'nightlight_sd', 'EVI', 'EVI_stdDev',  'FAO_price', 'Evap_tavg_mean', 'Evap_tavg_stdDev', 'LWdown_f_tavg_mean', 'LWdown_f_tavg_stdDev', 'Lwnet_tavg_mean', 'Lwnet_tavg_stdDev', 'Psurf_f_tavg_mean', 'Psurf_f_tavg_stdDev', 'Qair_f_tavg_mean', 'Qair_f_tavg_stdDev', 'Qg_tavg_mean', 'Qg_tavg_stdDev', 'Qh_tavg_mean', 'Qh_tavg_stdDev', 'Qle_tavg_mean', 'Qle_tavg_stdDev', 'Qs_tavg_mean', 'Qs_tavg_stdDev', 'Qsb_tavg_mean', 'Qsb_tavg_stdDev', 'RadT_tavg_mean', 'RadT_tavg_stdDev', 'Rainf_f_tavg_mean', 'Rainf_f_tavg_stdDev', 'SnowCover_inst_mean', 'SnowCover_inst_stdDev', 'SnowDepth_inst_mean', 'SnowDepth_inst_stdDev', 'Snowf_tavg_mean', 'Snowf_tavg_stdDev', 'SoilMoi00_10cm_tavg_mean', 'SoilMoi00_10cm_tavg_stdDev', 'SoilMoi10_40cm_tavg_mean', 'SoilMoi10_40cm_tavg_stdDev', 'SoilMoi100_200cm_tavg_mean', 'SoilMoi100_200cm_tavg_stdDev', 'SoilMoi40_100cm_tavg_mean', 'SoilMoi40_100cm_tavg_stdDev', 'SoilTemp00_10cm_tavg_mean', 'SoilTemp00_10cm_tavg_stdDev', 'SoilTemp10_40cm_tavg_mean', 'SoilTemp10_40cm_tavg_stdDev', 'SoilTemp100_200cm_tavg_mean', 'SoilTemp100_200cm_tavg_stdDev', 'SoilTemp40_100cm_tavg_mean', 'SoilTemp40_100cm_tavg_stdDev', 'SWdown_f_tavg_mean', 'SWdown_f_tavg_stdDev', 'SWE_inst_mean', 'SWE_inst_stdDev', 'Swnet_tavg_mean', 'Swnet_tavg_stdDev', 'Tair_f_tavg_mean', 'Tair_f_tavg_stdDev', 'Wind_f_tavg_mean', 'Wind_f_tavg_stdDev', 'gpp_sd', 'gpp_mean', 'CPI', 'GDP', 'CC', 'gini', 'WFP_Price', 'WFP_Price_std']
time_variants_m12 = [variant + '_m12' for variant in time_variants]
time_variants_list = time_variants + time_variants_m12
l2_index = []
# if columns in time_variants_list, record its index and restore in l2_index
for i, col in enumerate(df.columns):
    if col in time_variants_list:
        l2_index.append(i)





Using features for clustering: ['WFP_Price_std']


C:\Users\swl00\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:1389: ConvergenceWarning: Number of distinct clusters (20) found smaller than n_clusters (100). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)


Created 100 clusters from 4081 unique admin codes
Cluster sizes range from 1 to 1532 admin codes


In [5]:
# l1 index is all columns except l2 index
l1_index = [i for i in range(df.shape[1]) if i not in l2_index]

X = df.values
X = comp_impute(X)

Checking column 0 for inf values
Checking column 1 for inf values
Checking column 2 for inf values
Checking column 3 for inf values
Checking column 4 for inf values
Checking column 5 for inf values
Checking column 6 for inf values
Checking column 7 for inf values
Checking column 8 for inf values
Checking column 9 for inf values
Checking column 10 for inf values
Checking column 11 for inf values
Checking column 12 for inf values
Checking column 13 for inf values
Checking column 14 for inf values
Checking column 15 for inf values
Checking column 16 for inf values
Checking column 17 for inf values
Checking column 18 for inf values
Checking column 19 for inf values
Checking column 20 for inf values
Checking column 21 for inf values
Checking column 22 for inf values
Checking column 23 for inf values
Checking column 24 for inf values
Checking column 25 for inf values
Checking column 26 for inf values
Checking column 27 for inf values
Checking column 28 for inf values
Checking column 29 for i

In [6]:
nowcasting = False
max_depth = None
results_df= pd.DataFrame(columns=['year','precision(0)', 'recall(0)', 'f1(0)', 'precision(1)', 'recall(1)', 'f1(1)', 'precision_base(0)', 'recall_base(0)', 'f1_base(0)', 'precision_base(1)', 'recall_base(1)', 'f1_base(1)',
                                  'num_samples(0)','num_samples(1)'])

y_pred_test = pd.DataFrame(columns=['year', 'month','adm_code','fews_ipc_crisis_pred','fews_ipc_crisis_true'])
#train test split


In [7]:

# TODO: two layer model not yet completed
# TODO: why some f1, recall, and sensitivity is higher in the base model?
# Updated main notebook code with consistent 2-layer evaluation

if nowcasting:
    for year in range(2021,2025):
        # Train-test split
        (Xtrain, ytrain, Xtrain_loc, Xtrain_group,
        Xtest, ytest, Xtest_loc, Xtest_group) = train_test_split_by_year(
            X, y, X_loc, X_group, df['years'].values, test_year=year)
        ytrain = ytrain.astype(int)
        ytest  = ytest.astype(int)
        
        # Split features for both layers
        Xtrain_L1 = Xtrain[:, l1_index]
        Xtrain_L2 = Xtrain[:, l2_index]
        Xtest_L1 = Xtest[:, l1_index]
        Xtest_L2 = Xtest[:, l2_index]
        
        # Create and train 2-layer GeoRF model
        georf_2layer = GeoRF(
            min_model_depth=MIN_DEPTH,
            max_model_depth=MAX_DEPTH, 
            n_jobs=N_JOBS,
            max_depth=max_depth
        )
        
        # Train 2-layer model
        georf_2layer.fit_2layer(Xtrain_L1, Xtrain_L2, ytrain, Xtrain_group, val_ratio=VAL_RATIO)
        
        # Get predictions
        ypred = georf_2layer.predict_2layer(Xtest_L1, Xtest_L2, Xtest_group, correction_strategy='flip')
        
        # Evaluate: This now compares 2-layer GeoRF vs 2-layer Base RF
        (pre, rec, f1, pre_base, rec_base, f1_base) = georf_2layer.evaluate_2layer(
            X_L1_test=Xtest_L1,
            X_L2_test=Xtest_L2, 
            y_test=ytest,
            X_group_test=Xtest_group,
            X_L1_train=Xtrain_L1,
            X_L2_train=Xtrain_L2,
            y_train=ytrain, 
            X_group_train=Xtrain_group,
            correction_strategy='flip',
            print_to_file=True
        )
        
        # Calculate weighted averages (same as your forecasting model)
        nsample_class = np.bincount(ytest)
        
        # Store results
        results_df = pd.concat([results_df, pd.DataFrame({
            'year': year,
            'precision(0)': pre[0],
            'precision(1)': pre[1],
            'recall(0)': rec[0],
            'recall(1)': rec[1],
            'f1(0)': f1[0],
            'f1(1)': f1[1],
            'precision_base(0)': pre_base[0],
            'precision_base(1)': pre_base[1],
            'recall_base(0)': rec_base[0], 
            'recall_base(1)': rec_base[1],
            'f1_base(0)': f1_base[0],
            'f1_base(1)': f1_base[1],
            'num_samples(0)': nsample_class[0],
            'num_samples(1)': nsample_class[1]
        }, index=[0])], ignore_index=True)
        
        y_pred_test = pd.concat([y_pred_test, pd.DataFrame({
            'year': year, 
            'month': Xtest[:,1], 
            'adm_code': Xtest[:,0], 
            'fews_ipc_crisis_pred': ypred, 
            'fews_ipc_crisis_true': ytest
        })], ignore_index=True)
        
        print(f"Year {year} - 2-Layer GeoRF F1: {f1:.4f}, 2-Layer Base RF F1: {f1_base:.4f}")
    pred_test_name = 'y_pred_test_g'
    results_df_name = 'results_df_g'
    if assignment=='polygons':
        pred_test_name = pred_test_name + 'pn'
        results_df_name = results_df_name + 'pn'
    elif assignment == 'grid':
        pred_test_name = pred_test_name + 'gn'
        results_df_name = results_df_name + 'gn'
    elif assignment == 'country':
        pred_test_name = pred_test_name + 'cn'
        results_df_name = results_df_name + 'cn'
    elif assignment == 'AEZ':
        pred_test_name = pred_test_name + 'aen'
        results_df_name = results_df_name + 'aen'
    elif assignment == 'country_AEZ':
        pred_test_name = pred_test_name + 'caen'
        results_df_name = results_df_name + 'caen'
    elif assignment == 'geokmeans':
        pred_test_name = pred_test_name + 'gkn'
        results_df_name = results_df_name + 'gkn'
    elif assignment == 'all_kmeans':
        pred_test_name = pred_test_name + 'akn'
        results_df_name = results_df_name + 'akn'
    else:
        raise ValueError(f"Unknown assignment method: {assignment}")

    if max_depth is not None:
        pred_test_name = pred_test_name + f'_d{max_depth}.csv'
        results_df_name = results_df_name + f'_d{max_depth}.csv'
    else:
        pred_test_name = pred_test_name + '.csv'
        results_df_name = results_df_name + '.csv'
    y_pred_test.to_csv(pred_test_name, index=False)
    results_df.to_csv(results_df_name, index=False)

else:
    # Your existing forecasting code remains exactly the same
    for year in range(2021,2025):
        (Xtrain, ytrain, Xtrain_loc, Xtrain_group,
        Xtest, ytest, Xtest_loc, Xtest_group) = train_test_split_by_year(
            X, y, X_loc, X_group, df['years'].values, test_year=year)
        ytrain = ytrain.astype(int)
        ytest  = ytest.astype(int)
        
        georf = GeoRF(min_model_depth = MIN_DEPTH, max_model_depth = MAX_DEPTH, n_jobs = N_JOBS, max_depth=max_depth)
        georf.fit(Xtrain, ytrain, Xtrain_group, val_ratio = VAL_RATIO)
        ypred = georf.predict(Xtest, Xtest_group)
        
        # This returns single-layer GeoRF vs single-layer Base RF
        (pre, rec, f1, pre_base, rec_base, f1_base) = georf.evaluate(Xtest, ytest, Xtest_group, eval_base = True, print_to_file = True)
        
        nsample_class = np.bincount(ytest)
        
        results_df = pd.concat([results_df, pd.DataFrame({
            'year': year,
            'precision(0)': pre[0],
            'precision(1)': pre[1],
            'recall(0)': rec[0],
            'recall(1)': rec[1],
            'f1(0)': f1[0],
            'f1(1)': f1[1],
            'precision_base(0)': pre_base[0],
            'precision_base(1)': pre_base[1],
            'recall_base(0)': rec_base[0], 
            'recall_base(1)': rec_base[1],
            'f1_base(0)': f1_base[0],
            'f1_base(1)': f1_base[1],
            'num_samples(0)': nsample_class[0],
            'num_samples(1)': nsample_class[1]
        }, index=[0])], ignore_index=True)
        
        y_pred_test = pd.concat([y_pred_test, pd.DataFrame({
            'year': year, 
            'month': Xtest[:,1], 
            'adm_code': Xtest[:,0], 
            'fews_ipc_crisis_pred': ypred, 
            'fews_ipc_crisis_true': ytest
        })], ignore_index=True)
        
    pred_test_name = 'y_pred_test_g'
    results_df_name = 'results_df_g'
    if assignment=='polygons':
        pred_test_name = pred_test_name + 'pn'
        results_df_name = results_df_name + 'pn'
    elif assignment == 'grid':
        pred_test_name = pred_test_name + 'gn'
        results_df_name = results_df_name + 'gn'
    elif assignment == 'country':
        pred_test_name = pred_test_name + 'cn'
        results_df_name = results_df_name + 'cn'
    elif assignment == 'AEZ':
        pred_test_name = pred_test_name + 'aen'
        results_df_name = results_df_name + 'aen'
    elif assignment == 'country_AEZ':
        pred_test_name = pred_test_name + 'caen'
        results_df_name = results_df_name + 'caen'
    elif assignment == 'geokmeans':
        pred_test_name = pred_test_name + 'gkn'
        results_df_name = results_df_name + 'gkn'
    elif assignment == 'all_kmeans':
        pred_test_name = pred_test_name + 'akn'
        results_df_name = results_df_name + 'akn'
    else:
        raise ValueError(f"Unknown assignment method: {assignment}")

    if max_depth is not None:
        pred_test_name = pred_test_name + f'_d{max_depth}.csv'
        results_df_name = results_df_name + f'_d{max_depth}.csv'
    else:
        pred_test_name = pred_test_name + '.csv'
        results_df_name = results_df_name + '.csv'
    y_pred_test.to_csv(pred_test_name, index=False)
    results_df.to_csv(results_df_name, index=False)

model_dir: result_GeoRF_43


c:\Users\swl00\IFPRI Dropbox\Weilun Shi\Google fund\Analysis\2.source_code\Step5_Geo_RF_trial\Food_Crisis_Cluster\helper.py:235: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  s_branch[''][:gid_list.shape[0]] = gid_list
c:\Users\swl00\IFPRI D

model_dir: result_GeoRF_43
Printing to file.


C:\Users\swl00\AppData\Local\Temp\ipykernel_22172\939127335.py:132: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, pd.DataFrame({
C:\Users\swl00\AppData\Local\Temp\ipykernel_22172\939127335.py:150: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  y_pred_test = pd.concat([y_pred_test, pd.DataFrame({


model_dir: result_GeoRF_44


...

model_dir: result_GeoRF_44
Printing to file.
model_dir: result_GeoRF_45


c:\Users\swl00\IFPRI Dropbox\Weilun Shi\Google fund\Analysis\2.source_code\Step5_Geo_RF_trial\Food_Crisis_Cluster\helper.py:235: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  s_branch[''][:gid_list.shape[0]] = gid_list
c:\Users\swl00\IFPRI D

model_dir: result_GeoRF_45
Printing to file.
model_dir: result_GeoRF_46


c:\Users\swl00\IFPRI Dropbox\Weilun Shi\Google fund\Analysis\2.source_code\Step5_Geo_RF_trial\Food_Crisis_Cluster\helper.py:235: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  s_branch[''][:gid_list.shape[0]] = gid_list
c:\Users\swl00\IFPRI D

model_dir: result_GeoRF_46
Printing to file.
